<div align="right"> <h1 align="right">Anhang 3</h1></div>

<div align="center"> <h1 align="center">Erstellung eines Netzwerkgraphen aus Personendaten der Hamburg-<br/>Bibliographie mit der Python-Bibliothek Pyvis.</h1></div>

<br/>

<div align="left"> <h2 align="left">A. Der Weg von der Ausgangsdatei zur json-Datei für die Nutzung in Pyvis.</h2></div>

Dieses Skript ist im Zusammenhang mit der Ausarbeitung für dieses Projekt zu lesen, Angaben zu Datenstruktur der Ausgangsdatei sowie Transformationsanforderungen im allgemeinen finden sich auch dort im Text und in den Anhängen.

### 1. Importieren der nötigen Bibliotheken¶

In [1]:
from pyvis.network import Network
import random
import json

### 2. Aufrufen der gelieferten Ausgangsdatei

In [2]:
with open ("hhb_td_2022-01-19.pp", "r+", encoding = "utf-8") as infile:
    file = infile.readlines()

### 3. Wandlung der Lines in einzelne Dictionaries, Teilung der Lines in Keys und Values
Die einzelnen Zeilen der Ausgangsdatei teilen sich in Feldbezeichnung und Inhalt, die Feldbezeichnung ist durch ein Leerzeichen von dem Inhalt des Feldes getrennt. Diese Struktur kann dazu genutzt werden, eine Liste "entries" mit Key-Value-Paaren als Dictionaries für jede Zeile herzustellen. Dabei wird die Feldbenennung zum Key, der Feldinhalt zum Value der Dictionaries. Entstehen wird die Liste von Dictionaries "entries".

In [3]:
entries = []
    
for line in file:
    
    # Das "*" vermeidet einen ValueError bei der Übertragung der Strings in eine Liste beim Splitten
    dict_line = {}
    key, *value = line.strip().replace("\x1f", " ").split(" ", 1)
    dict_line[key] = value
    
    entries.append(dict_line)

### 4. Gesammelte Bearbeitung der Values von Key "041R" 
Die Entscheidung, den Beginn der Values der Keys "041R" an dieser Stelle schon endzubearbeiten, hat praktische Gründe: Die Values der Keys "041R" enthalten Daten, die später bei Erstellung des Graphen mit dem Value von Key "041A" (Person des Datensatzes) relationiert werden müssen. Sie sind in den Originaldaten mehrfach vorhanden mit unterschiedlichen Value-Inhalten, die erhalten bleiben müssen (vgl. Anhang 1). Für die Zusammenfassung zu Dictionaries müssen sie individualisiert werden, damit sie nicht überschrieben werden. Um eine spätere Einzelbehandlung der dann individualisierten Keys zu vermeiden und diese in einem Rutsch zu erledigen, geschieht dies an dieser Stelle vor der Individualisierung. Die gleiche Datenstruktur der Values begünstigt dies. Das Strippen bzw. Replacen des Beginns der Values wird von links bis zum Subfield des Inhaltes durchgeführt. Restlich zu löschende Codierungen rechts vom Inhalt werden in späteren Arbeitschritten erledigt, die Codierungen werden noch als Anker zum Vereinigen der Inhalte benötigt.

In [4]:
for item in entries:
    for key in item:
        item[key] = str(item[key]).strip("[' ").strip("']").strip("\\").replace("\"", "'")
        
        if "041R" in item:
            item[key] = item[key].lstrip(
                "[0123456789]X").lstrip().lstrip(
                "VTdv").lstrip().lstrip(
                "6gnd\/[0123456789]-[0123456789]X").lstrip().lstrip(
                "S[pkcsgtfz]").lstrip().replace("a", "", 1)

### 5. Individualisierung gleichnamiger Keys I: 
Umbenennung der verschiedenen Keys des Feldes "041R" und der Keys "060R", die mehrfach vorkommen können, anhand der Codierungen in den Inhalten ihrer Values. Die Codierungen, die den Inhalt definieren, werden als Keyergänzung angehängt:

    Key "041R": "4beru" = Berufsangaben
                "4bezf" = familiäre Beziehung
                "4bezb" = berufliche Beziehung
                "4ort"  = Geburts-/Sterbeort
                "4affi" = institutionelle Affiliation/Zugehörigkeit
    Key "060R": "4datl" = Lebensdaten
                "4datx" = exakte Lebensdaten


In [5]:
for item in entries:
    for key, value in list(item.items()):
        
        if "041R" == key:
            if "4beru" in value:
                item["041R_beru"] = item.pop("041R")    
                
            if "4bezf" in value:
                item["041R_bezf"] = item.pop("041R") 
                
            if "4bezb" in value:
                item["041R_bezb"] = item.pop("041R") 
                
            if "4ort" in value:
                item["041R_ort"] = item.pop("041R") 
        
            if "4affi" in value:
                item["041R_affi"] = item.pop("041R")        
                
        if "060R" == key:                             
            if "4datl" in value:
                item["060R_datl"] = item.pop("060R")
                             
            if "4datx" in value:
                item["060R_datx"] = item.pop("060R")

### 6. Individualisierung gleichnamiger Keys II
Nach dem ersten Individualisierungsschritt für die Keys "041R" und "060R" braucht es einen zweiten, denn noch ist nicht vermieden, dass Keys doppelt vorhanden sind. Dazu erhalten die gefährdeten Keys einen durchlaufenden Nummernzusatz und werden so individualisiert.

In [6]:
i=0

for item in entries:
    
    if "041R_beru" in item:
        item[str("041R_beru-" + str(i))] = item.pop("041R_beru")
              
    if "041R_bezf" in item:
        item[str("041R_bezf-" + str(i))] = item.pop("041R_bezf")
        
    if "041R_bezb" in item:
        item[str("041R_bezb-" + str(i))] = item.pop("041R_bezb")
        
    if "041R_ort" in item:
        item[str("041R_ort-" + str(i))] = item.pop("041R_ort")
        
    if "041R_affi" in item:
        item[str("041R_affi-" + str(i))] = item.pop("041R_affi")
  
    i=i+1    

### 7. Zusammenfassung der zu einem Personendatensatz gehörigen Einzel-Dictionaries zu gruppierten Dictionaries
Die einzelnen Zeilen-Dictionaries werden nun zu gruppierten Items einer List of Dictionaries transformiert, bei der nicht mehr jedes Item einer Zeile der Ausgangsdatei entspricht, sondern dem Datensatz einer Person. Dabei dient das Feld "001A" als Anker, dessen Dictionary anzeigt, dass die folgenden Dictionaries, bis zum zum nächsten Vorkommnis des Feldes "001A" zu einem Personendatensatz gehören. Entstehen wird neu die Liste mit Dictionaries "grouped_dicts".

In [7]:
grouped_dicts = []
current_entry = {}
    
for item in entries:    
    if "001A" in item:
        
        # erzeugt ein Dictionary für die Keys eines jeden Datensatzes, wenn das if-Statement wahr ist   
        current_entry = {} 
        
        # fügt das dict "current_entry" in die Liste "grouped_dicts" an
        grouped_dicts.append(current_entry)
    
    # liest die Einzeldictionaries aus "entries" nach "current_entry" in "grouped_dicts"
    current_entry.update(item)             

In [8]:
# Zählung Gesamtmenge der Datensätze der Ausgangdatei
count_items = 0
for item in grouped_dicts:
    if "003@" in item.keys():
        count_items += 1
count_items        

72125

### 8. Vereinigung zusammengehöriger Values unter einen Key
Durch das Individualisieren der gleichnamigen Keys wurden deren Values gesichert. In einem nächsten Schritt müssen die Values der einzelnen Keys nun in einen Gesamtvalue zusammengefügt werden. Für diese Filterung werden wieder die oben erwähnten Codierungen genutzt. Um unterschiedliche Werte gleichnamiger Keys in einen Wert zu integrieren, werden neue Values generiert und die alten gelöscht. Danach erfolgt die Löschung der alten, durchnummerierten Key/Value-Paare.

In [9]:
for item in grouped_dicts:
    
    item["joined_val-041R_beru"] = []
    item["joined_val-041R_bezf"] = []
    item["joined_val-041R_bezb"] = []
    item["joined_val-041R_ort"] = []
    item["joined_val-041R_affi"] = []
    
    for key, value in item.items():                                                        
        if "041R_beru-" in key:
            item["joined_val-041R_beru"].append(value)
            
        if "041R_bezf-" in key:
            item["joined_val-041R_bezf"].append(value)
            
        if "041R_bezb-" in key:
            item["joined_val-041R_bezb"].append(value)
            
        if "041R_ort-" in key:
            item["joined_val-041R_ort"].append(value)
            
        if "041R_affi-" in key:
            item["joined_val-041R_affi"].append(value)
            
    # Löschung der alten Key/Value-Paare
    old_keys = ["041R_beru-", "041R_bezf-", "041R_bezb-", "041R_ort-", "041R_affi-"]
    
    for key in item.copy():
        for element in old_keys:
            if element in key:
                item.pop(key)
      
    # Rückbenennung der Keys 
        if key == "joined_val-041R_beru":                                                
            item["041R_beru"] = item.pop("joined_val-041R_beru")
        
        if key == "joined_val-041R_bezf":
            item["041R_bezf"] = item.pop("joined_val-041R_bezf")
            
        if key == "joined_val-041R_bezb":
            item["041R_bezb"] = item.pop("joined_val-041R_bezb")

        if key == "joined_val-041R_ort":
            item["041R_ort"] = item.pop("joined_val-041R_ort")
            
        if key == "joined_val-041R_affi":
            item["041R_affi"] = item.pop("joined_val-041R_affi")

### 9. Aufräumen I: leere Values löschen

In [10]:
for item in grouped_dicts:
    item.pop("", None)

In [11]:
for item in grouped_dicts:
    for key, value in list(item.items()):
        if not value:
            del item[key]

### 10. Aufräumen II: überflüssige Satzarten löschen
Die Ausgangsdatei beinhaltete sämtliche Normdatensätze der Hamburg-Bibliographie, nicht nur Personen. Im nächsten Schritt werden alle Items, die keine Personennormdaten verzeichnen, gelöscht. Um folgende Satzarten handelt es sich, sie werden mittels eindeutigen Zeichenabfolgen in den Values des Keys "041A" gegriffen.

    Zeichenfolge: "Sk a" = Körperschaftsdaten
                  "St a" = Titeldaten
                  "Sf a" = Medienartdaten
                  "Sg a" = Gebietsdaten
                  "Ss a" = Sachdaten
                  "Sz a" = Zeitdaten
                  "Sc a" = Gebietskörperschaftsdaten

In [12]:
for item in grouped_dicts.copy():
    
    data_type = ["Sk a", "St a", "Sf a", "Sg a", "Ss a", "Sz a", "Sc a"]
    for key in item:
        if "041A" == key:
            for character in data_type:
                if character in item[key]:
                    grouped_dicts.remove(item)

Für die nächsten Filterungen der Datenatztypen, die keine (relevanten) Personendaten verzeichenen, wie Datensätze von Schlagwortketten, Personendatensätze von Nicht-Hamburgern sowie Datensätzen von Familien, muss bei meinen Programmierkenntnissen einzeln iteriert werden, da die Filterkriterien nicht ausschließlich einzeln in den Values vorkommen können. Folgende Filterkriterien befinden sich in den Values der Keys "041A" und "050D".

    Filterkriterien: "Sp a" und " x"           = Datensätze von Schlagwortketten
                     "Sp a" und " gFamilie"    = Familiennormdaten
                     "Sp a" und "<Familie>"    = Familiennormdaten
                     "abio-"                   = Personennormdaten von Nicht-Hamburgern

In [13]:
for item in grouped_dicts.copy():
    for key in item:
        if key == "041A":
            if "Sp a" in item[key]:
                if " x" in item[key]:
                    grouped_dicts.remove(item)

In [14]:
for item in grouped_dicts.copy():
    for key in item:
        if key == "041A":
            if "Sp a" in item[key]:
                if " gFamilie" in item[key]:
                    grouped_dicts.remove(item)

In [15]:
for item in grouped_dicts.copy():
    for key in item:
        if key == "041A":
            if "Sp a" in item[key]:
                if "<Familie>" in item[key]:
                    grouped_dicts.remove(item)

In [16]:
for item in grouped_dicts.copy():
    for key in item:
        if "050D" == key:
            if "abio-" in item[key]:
                grouped_dicts.remove(item)

In [17]:
# Zählung der Gesamtmenge der verzeichneten Personendaten in der Hamburg-Bibliographie
count_items = 0
for item in grouped_dicts:
    if "003@" in item.keys():
        count_items += 1
count_items        

19869

### 11. Aufräumen III: überflüssige Key/Value-Paare löschen
Anmerkung zu Feld 041R: es kann gelöscht werden, da die relevanten Inhalte in neue neubenannte Key/Value-Paare überführt wurden (Feld "041R" ist ein Sammelfeld für viele relationierte Daten, die jeweils durch Codierungen - weitaus mehr als die hier relevanten sind möglich - inhaltlich zugeordnet wurden, vgl. Anhang 1). Es befinden sich also nur noch irrelevante Inhalte im Feld sowie relevante Inhalte, die nicht zugeordnet werden können, weil bei ihnen die Codierung nicht aussagekräftig ist, die Codierung fehlt oder diese unkorrekt eingegeben wurde. Auf diese relevanten Inhalte könnte erst nach einer händischen Umarbeitung der originalen Ausgangsdaten zugegriffen werden.

In [18]:
for item in grouped_dicts:
    
    item.pop("001A", None)
    item.pop("001B", None)
    item.pop("001D", None)
    item.pop("001U", None)
    item.pop("001X", None)
    item.pop("002@", None)
    item.pop("041@", None)
    item.pop("041P", None)
    item.pop("047A/53", None)
    item.pop("050C", None)
    item.pop("050G", None)
    item.pop("006Y", None)
    item.pop("047A/04", None)
    item.pop("050E", None)
    item.pop("003D", None)
    item.pop("041O", None)
    item.pop("050H", None)
    item.pop("041R", None)  # Löschung des Keys (Erklärung siehe vorige Zelle)
    item.pop("007K", None)

Im nächsten Abschnitt werden die Values der übrig gebliebenen Keys endbehandelt, d.h. von allen restlichen Codierungen befreit, bei Bedarf gestript, replaced, gesliced, gesplittet u.a., so dass der reine Inhalt übrig bleibt. Neben des Präsentationszweckes im Graphen und der Verhinderung von Whitespace-Fehlermeldungen ist dieser Schritt auch besonders wichtig für die Values der Keys "041A" und "041R_beru" bzw. "041R_bezf", die bei der Generierung des Graphen direkt abgeglichen werden und daher deren Inhalte in beiden Values gleichlauten müssen. Die Values der übrigen Keys für relationierte Angaben "041R_bezb" und "041R_affi" werden mitbehandelt für eine eventuelle spätere Nutzung.  Weiterhin werden, um Fehlermeldungen bei der Erzeugung des Graphen zu vermeiden, für alle Keys leere Values erzeugt, sollten in den Originaldaten keine Inhalte vorhanden gewesen sein, damit in jedem Dictionary ein Wert angesteuert werden kann. Dies gilt nicht für den Key "041A". Um dort Fehlermeldungen durch fehlenden Values aufgrund falsch eingegebener Originaldaten zu vermeiden, werden in diesem Fall keine leeren Values erzeugt, sondern die Dictionaries ganz gelöscht, da ohne einen Inhalt für den Ankerkey "041A" diese Person quasi ja gar nicht vorhanden ist und ein leerer Node für den Graphen generiert werden würde.
    Im Folgenden wird aufgrund der Fülle nicht die Gesamtheit aller zu entfernenden Codierungen in den jeweiligen Values näher erläutert, dazu empfiehlt sich bei Interesse ein Blick in die Ausgangsdaten.

### 12. Abschlussbehandlung Value "041A" (Personenname)

In [19]:
for item in grouped_dicts:
    for key in item:
        if key == "041A":
            item[key] = str(item[key][4:]).lstrip().replace("^", "").replace(" g", "; ")
            item[key] = [item[key]]

In [20]:
for item in grouped_dicts.copy():
    if "041A" not in item:
        grouped_dicts.remove(item)   

### 13. Abschlussbehandlung Value "047A/02" (Biogramm)

In [21]:
for item in grouped_dicts:
    for key in item:
        if key == "047A/02":
            item[key] = str(item[key]).replace("a", "", 1)
            item[key] = [item[key]]

In [22]:
for item in grouped_dicts:
    for key in item.copy():
        if "047A/02" not in item:
            item["047A/02"] = []

### 14. Abschlussbehandlung Value "047A" (Berufsangaben Altdaten)
Aufgrund von Altdatentransformationen existieren auch Berufsangaben in einem anderen Feld als dem richtlinienvorgeschriebenen "041R". Glücklicherweise können nicht beide Felder mit Berufsangaben in einem Datensatz der Ausgangsdaten vorkommen, es gibt sie nur je und je, das hat Arbeitsschritte gespart. "047A" wird hier bearbeitet und zum Key "041R_beru" gewandelt.

In [23]:
for item in grouped_dicts:
    for key in item:
        if key == "047A":
            item[key] = str(item[key]).lstrip(
                "a").replace(
                ".", "").replace(
                "; ", "").replace(
                "und ", "").replace(
                ",", "").split()

In [24]:
for item in grouped_dicts:
    if "047A" in item:
        item["047A_beru"] = item.pop("047A")

### 15. Abschlussbehandlung Value "041R_beru" (Berufsangaben)

In [25]:
for item in grouped_dicts:
    for key in item.copy():  
        if key == "041R_beru":
            item["temp_041R_beru"] = []
            for element in item[key]:
                item["temp_041R_beru"].append(element.replace(" 4beru", "").replace(" v", ", ", 1))
                
            item.pop(key)

In [26]:
for item in grouped_dicts:
    if "temp_041R_beru" in item:
        item["041R_beru"] = item.pop("temp_041R_beru")

Umbenennen des Altdatenfeldes "047A" in das gleichnamige Feld "041R_beru", damit alle Berufsdaten die gleiche Keybenennung haben (siehe oben).

In [27]:
for item in grouped_dicts:
    for key in item.copy(): 
        if key == "047A_beru":
            item["041R_beru"] = item.pop("047A_beru")
        
        if "041R_beru" not in item:
                item["041R_beru"] = []

### 16. Abschlussbehandlung Value "041R_bezf" (familiäre Beziehungsangaben)
Eine Besonderheit für die Inhalte von Key "041R_bef" (familiäre Beziehungsangaben) sind erwähnenswert: in den Originaldaten können neben den Namen der Familienmitglieder auch die jeweilige Beziehungskategorie (Schwester, Vater, Sohn u.a.) erfasst werden. Leider darf diese nicht erhalten bleiben, da zum Abgleich mit Key "041A" bei der Generation des Graphen gleichlautende Einträge vorhanden sein müssen (vgl. oben). Das ist schade, denn so gehen Informationen verloren. In einer Ausbaustufe des Projektes ist geplant, diese Angaben in die Biogramme ("047A/02") zu integrieren, um sie zu erhalten.

In [28]:
for item in grouped_dicts:
    for key in item.copy():  
        if key == "041R_bezf":
            item["temp_041R_bezf"] = []
            for element in item[key]:
                element = element.rsplit("4bezf", 1)[0]
                element = element.rstrip("4bezf").rstrip().replace("^", "")
                item["temp_041R_bezf"].append(element)
                
            item.pop(key)

In [29]:
for item in grouped_dicts:
    for key in item.copy():
        if key == "temp_041R_bezf":
            item["041R_bezf"] = item.pop("temp_041R_bezf") 
        
        if "041R_bezf" not in item:
            item["041R_bezf"] = []

### 17. Abschlussbehandlung Value "041R_bezb" (berufliche Beziehungsangaben)
Vgl. Punkt 16. Diese Angaben gehören zu dem Kreis der für den Graphen potentiell nutzbaren. Da sie in den Originaldaten jedoch nicht häufig besetzt sind, wird hier zunächst davon Abstand genommen.

In [30]:
for item in grouped_dicts:
    for key in item.copy():  
        if key == "041R_bezb":
            item["temp_041R_bezb"] = []
            for element in item[key]:
                element = element.rsplit("4bezb", 1)[0]
                element = element.rstrip("4bezb").rstrip().replace("^", "")
                item["temp_041R_bezb"].append(element)
                
            item.pop(key)

In [31]:
for item in grouped_dicts:
    for key in item.copy():
        if key == "temp_041R_bezb":
            item["041R_bezb"] = item.pop("temp_041R_bezb")
        
        if "041R_bezb" not in item:
                item["041R_bezb"] = []

### 18. Abschlussbehandlung Value "041R_affi" (institutionelle Beziehungsangaben)
Vgl. Punkt 17.

In [32]:
for item in grouped_dicts:
    for key in item.copy():  
        if key == "041R_affi":
            
            item["temp_041R_affi"] = []
            for element in item[key]:
                item["temp_041R_affi"].append(
                    element.replace(
                    " 4affi v", "; ").replace(
                    " 4affi", "").replace(
                    " Z", " ").replace(
                    " g", ", ").replace(
                    " x", ", "))
            item.pop(key)

In [33]:
for item in grouped_dicts:
    for key in item.copy():
        
        if key == "temp_041R_affi":
            item["041R_affi"] = item.pop("temp_041R_affi")
        
        if "041R_affi" not in item:
                item["041R_affi"] = []

Im folgenden werden Angaben endbehandelt, die zu den biographischen Angaben zählen (Geburts-/Sterbeorte, Lebensdaten). Sie sind für eine Ausbaustufe des Projektes zur Integration in die Biogramme eingeplant. Dafür werden ggf. auch schon Angaben hinzugefügt.

### 19. Abschlussbehandlung Value "041R_ort" (Geburts-/Sterbeort)

In [34]:
for item in grouped_dicts:
    for key in list(item):  
        if key == "041R_ort":
            item["temp_041R_ort"] = []
            for element in list(item[key]):
                if "4ortg" in element:
                    item["temp_041R_ort"].append("geb. " + element.replace(" 4ortg", ""))
                if "4orts" in element:
                    item["temp_041R_ort"].append("gest. " + element.replace(" 4orts", ""))
            item.pop(key)

In [35]:
for item in grouped_dicts:
    for key in item.copy():
        
        if key == "temp_041R_ort":
            item["041R_ort"] = item.pop("temp_041R_ort")
        
        if "041R_ort" not in item:
                item["041R_ort"] = []

### 20. Abschlussbehandlung Value "060R_datl" (einfache Lebensdaten)

In [36]:
for item in grouped_dicts:
    for key in item:
        if key == "060R_datl":
            item[key] = item[key].replace(" 4datl", "").replace(" v", ", ", 1).replace(" b", " +", 1)
            
            if item[key][0] == "a":
                item[key] = "*" + item[key][1:] 
            
            if item[key][0] == "b":
                item[key] = "+" + item[key][1:]
            
            item[key] = [item[key]]            

In [37]:
for item in grouped_dicts:
    for key in item.copy():
        if "060R_datl" not in item:
                item["060R_datl"] = []

### 21. Abschlussbehandlung Value "060R_datx" (exakte Lebensdaten)

In [38]:
for item in grouped_dicts:
    for key in item:
        if key == "060R_datx":
            item[key] = item[key].replace(" 4datx", "").replace(" v", ", ", 1).replace(" b", " +", 1)
            
            if item[key][0] == "a":
                item[key] = "*" + item[key][1:] 
            
            if item[key][0] == "b":
                item[key] = "+" + item[key][1:]
            
            item[key] = [item[key]]

In [39]:
for item in grouped_dicts:
    for key in item.copy():
        if "060R_datx" not in item:
                item["060R_datx"] = []

### 22. Abschlussbehandlung Value "060R" (falsch codierte Lebensaten / uncodierte Lebensdaten aus Altdaten)
In diesem Feld befinden sich nur noch, nachdem datl- und datx-codierte Angaben in eigene Key/Value-Paare gewandert sind, durch Eingabefehler falsch codierte datl-, datx-Daten sowie Daten ohne Codierung aus Altdaten. Es wird versucht, so viele Falschcodierungen zu löschen wie bei einem Datenscan gefunden werden konnten.

In [40]:
for item in grouped_dicts:
    for key in item:
        if key == "060R":
            item[key] = item[key].replace(
                " 4datw", "").replace(
                " 4dats", "").replace(
                " 4datb", "").replace(
                " bdatx", "").replace(
                " bdatl", "").replace(
                " datz", "").replace(
                " datl", "").replace(
                " 4dtl", "").replace(
                " v", ", ", 1).replace(
                " b", " +", 1)
            
            if item[key][0] == "a":
                item[key] = "*" + item[key][1:] 
            
            if item[key][0] == "b":
                item[key] = "+" + item[key][1:]
                
            if item[key][0] == "":
                item[key] = "*" + item[key][1:] 
                
            item[key] = [item[key]]

In [41]:
for item in grouped_dicts:
    for key in item.copy():
        if "060R" not in item:
                item["060R"] = []

### 23. Abschlussbehandlung Value "003@"
Der Value von Key "003@" ist die Datensatz-ID für jeden Personendatensatz. Für die Genererierung des Netzwerkgraphen ist sie nicht relevant, aus datenpraktischen Gründen wird sie aber mitgeführt, um etwa Fehler in den Originaldaten schnell lokalisieren zu können.

In [42]:
for item in grouped_dicts:
    for key in item:
        if key == "003@":
            item["003@"] = [item["003@"]] 

### 24. Keys bekommen sprechende Namen

In [43]:
for item in grouped_dicts:
    for key in item.copy():
        
        if key == "060R":
            item["date"] = item.pop("060R")
            
        if key == "060R_datl":
            item["date_simple"] = item.pop("060R_datl")
            
        if key == "060R_datx":
            item["date_exact"] = item.pop("060R_datx")
            
        if key == "041R_beru":
            item["job"] = item.pop("041R_beru")
            
        if key == "041R_bezf":
            item["relation_fam"] = item.pop("041R_bezf")
            
        if key == "041R_bezb":
            item["relation_job"] = item.pop("041R_bezb")
            
        if key == "041R_affi":
            item["relation_inst"] = item.pop("041R_affi")
            
        if key == "041R_ort":
            item["place_birth_death"] = item.pop("041R_ort")
            
        if key == "047A/02":
            item["bio"] = item.pop("047A/02")
            
        if key == "041A":
            item["name"] = item.pop("041A")
            
        if key == "003@":
            item["ppn"] = item.pop("003@")

### 25. Erstellung json-Dateien
#### 25.1 Gesamtdatei

In [44]:
with open("hhbib_persons.json", "w") as outfile:
    json.dump(grouped_dicts, outfile, indent=4, ensure_ascii=False)

#### 25.2 Gesamtdatei zur Erstellung des Graphen in Pyvis
Als darstellerische Variante für den Graphen kann ein übergreifendes Item "Hamburg" gedacht werden, das als "Urpunktnode" alle Personen miteinander verbindet. Damit kann ein Graph mit einem Bezugspunkt für alle relationierten Daten erstellt werden (vgl. Punkt 26). Dies ist als darstellerische Variante gedacht. Daher wird ein übergeordnetes Dictionary mit der Keybenennung "Hamburg" angelegt, das bei Bedarf bei der Erstellung des Graphen berücksichtigt werden kann.

In [45]:
hamburg_node = {"Hamburg": grouped_dicts}

In [46]:
# Überprüfung der Anzahl der Items = Personen
count_items = 0
for item in hamburg_node["Hamburg"]:
    if "ppn" in item.keys():
        count_items += 1
count_items

19865

In [47]:
with open("hh_persons.json", "w") as outfile:
    json.dump(hamburg_node, outfile, indent=4, ensure_ascii=False)

#### 25.3 Datei zur Darstellung der Beziehungskonstellation "Person/familäre Beziehung":
Probedurchläufe haben ergeben, dass durch die Berücksichtigung der Gesamtpersonendaten mit familiärem Bezug lange Ladezeiten für die Anzeige des Graphen entstehen. Daher wird die Datenmenge in einer Datei auch auf Personendaten mit mindestens zwei Verwandtschaftsbeziehungen reduziert.

In [48]:
with open("hhbib_persons.json", "r+", encoding = "utf-8") as infile:
    hh_persons_fam = json.load(infile)

In [49]:
# Anzahl der Items, in denen Verwandtschaftsbeziehungen vorliegen
count_items = 0
for item in hh_persons_fam:
    for key, value in item.items():
        if key == "relation_fam":
            if value:
                count_items += 1
count_items

571

In [50]:
for item in hh_persons_fam.copy():
    for key, value in item.items():
        if key == "relation_fam":
            if not value:
                hh_persons_fam.remove(item)

In [51]:
with open("hh_persons_fam_all.json", "w") as outfile:
    json.dump({"Hamburg" : hh_persons_fam}, outfile, indent=4, ensure_ascii=False)

In [52]:
for item in hh_persons_fam.copy():
    for key, value in item.items():
        if key == "relation_fam":
            if len(value) <= 1:
                hh_persons_fam.remove(item)

In [53]:
# Anzahl der Personen mit mindestens zwei Verwandtschaftsbeziehungen
count_items = 0
for item in hh_persons_fam:
    if "ppn" in item.keys():
        count_items += 1
count_items

191

In [54]:
with open("hh_persons_fam_2.json", "w") as outfile:
    json.dump({"Hamburg" : hh_persons_fam}, outfile, indent=4, ensure_ascii=False)

#### 25.4 Datei zur Darstellung der Beziehungskonstellation "Person/Berufsangaben"
Auch hier ist wie unter Punkt 25.3 eine Datenreduzierung nötig, jedoch noch drastischer. Es werden zwei Dateien erstellt, die eine mit der Begrenzung auf mindestens vier Berufsangaben, die andere auf mindestens fünf.

In [55]:
with open("hhbib_persons.json", "r+", encoding = "utf-8") as infile:
    hh_persons_job = json.load(infile)

In [56]:
# Gesamtanzahl der Items mit verzeichneten Berufsangaben
count_items = 0
for item in hh_persons_job:
    for key, value in item.items():
        if key == "job":
            if value:
                count_items += 1
count_items

16757

Beschränkung auf Personen mit mindestens vier Berufsangaben

In [57]:
for item in hh_persons_job.copy():
    for key, value in item.items():
        if key == "job":
            if len(value) <= 3:
                hh_persons_job.remove(item)

In [58]:
# Anzahl der Items mit mindestens vier verzeichneten Berufsangaben
count_items = 0
for item in hh_persons_job:
    if "ppn" in item.keys():
        count_items += 1
count_items

673

In [59]:
with open("hh_persons_job_4.json", "w") as outfile:
    json.dump({"Hamburg" : hh_persons_job}, outfile, indent=4, ensure_ascii=False)

Beschränkung auf Personen mit mindestens fünf Berufsangaben

In [60]:
for item in hh_persons_job.copy():
    for key, value in item.items():
        if key == "job":
            if len(value) <= 4:
                hh_persons_job.remove(item)

In [61]:
# Anzahl der Items mit mindestens vier verzeichneten Berufsangaben
count_items = 0
for item in hh_persons_job:
    if "ppn" in item.keys():
        count_items += 1
count_items

197

In [62]:
with open("hh_persons_job_5.json", "w") as outfile:
    json.dump({"Hamburg" : hh_persons_job}, outfile, indent=4, ensure_ascii=False)

## B. Erstellung des Netzwerkgraphen
Wie das Skript zur Erstellung der json-Datei ist auch dieses in Zusammenhang mit der Ausarbeitung zu lesen. Über die einfache Erzeugung eines Graphen hinaus, werden hier zwei darstellerische Optionen zusätzlich angeboten:
1. Da die Personen als Familienmitglieder zwar untereinander durch Edges verknüpft werden, aber nicht die Familien untereinander (das spiegeln die Originaldaten nicht wieder), entstehen Familiensubnetze. Um ein verbindendes Element herzustellen, was allen Familien gemeinsam ist, kam die Idee eines übergeordneten "Hamburg"-Nodes auf, das mit allen Familien mittels eines Edges verbunden ist. Dazu wurde eine Liste mit jeweils nur einem Mitglied einer Familie erstellt, deren Einträge im Skript zur Erzeugung der Edges eingesetzt werden (hamburg_edges.txt). Es ist auch eine Variante möglich, in der alle Personen (Nodes) mit diesem Hamburg-Node verbunden sind. Das ist allerdings zwar hübsch anzusehen, aber etwas unpraktisch, weil dann die Familienverbindungen visuell nur schwer erkennbar und auseinander zu klabüstern sind.
1. Zur besseren visuellen Erfassbarkeit der einzelnen Familiennetze entstand weiterhin die Idee, diese farblich unterschiedlich zu erzeugen. Dazu wurden zufällig generierte Hex-ColorCodes erzeugt und diese dann den Nodes zugewiesen.

    In der folgenden Skriptzelle sind die gesamten, ausprobierten Optionen zur Visualisierung des Netzwerkes dargestellt. Für die Generierung varianter Visualisierungen/html-Dateien damit - auch mit den oben erstellten verschiedenen Dateien - werden einige entsprechend ausgeblendet oder Werte geändert. Die Einzelskripte dazu werden in der gesonderten Datei "variant_visualization_options.ipynb" notiert und annotiert.

In [63]:
with open("hh_persons_fam_2.json", "r", encoding = "utf-8") as infile:
    
    data_fam_2 = json.load(infile)
    hhbib_data = data_fam_2["Hamburg"]
    hamburg_edges = []
    
    # Laden der Datei "hamburg_edges.txt" und Erzeugung der Liste für die "Hamburg-Edges"
    with open("hamburg_edges.txt", "r", encoding = "utf-8") as infile_2:
        for line in infile_2:
            hh_name = line[:-1]
            hamburg_edges.append(hh_name)
               
        # Erzeugung des Graphen
        g = Network(height="1000px", width="100%", bgcolor="#FBFCFC", font_color="black")
        
        # Erzeugung "Hamburg-Node"
        for hamburg in data_fam_2:
            g.add_node(hamburg, color="#2471A3", size=100)
               
        # Erzeugung von Variablen für Nodes, relationierte Nodes, Title der Nodes    
        for person in hhbib_data:
            name = person["name"][0]
            relation = person["relation_fam"]
            bio = person["bio"]
            
            # Erzeugung einer Variablen zur Zuweisung von zufällig generierten Hex-ColorCodes zu den Nodes
            random_colnumber = random.randint(0, 17687459)
            hex_number = str(hex(random_colnumber))
            hex_number = "#" + hex_number[2:]
            
            # Erzeugung der Hauptnodes
            g.add_node(name, color=hex_number, title=bio, size=50) 
            
            # Erzeugung der relationierten Nodes sowie den Edges
            for rela in relation:     
                g.add_node(rela, color=hex_number, size=50)
                g.add_edge(name, rela)
                
                # Erzeugung der "Hamburg-Edges"
                for item in hamburg_edges:
                    if item == name:
                        g.add_edge(hamburg, name)
                    if item == rela:
                        g.add_edge(hamburg, rela)
        
        # Layout-Algorithmen
        g.barnes_hut()        
        #g.force_atlas_2based()
        #g.hrepulsion()
        #g.repulsion()
        #g.show_buttons()
        #g.set_options(''' var options = {
        #"nodes": {"font": {"size": 20}},
        #"physics": {"barnesHut": {"springLength": 400, "springConstant": 0.1}}}  ''')
        
        # Erzeugung der Visualisierung und der html-Datei
        g.show("hhbib_networkGraph_1.html")